In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
train_data = pd.read_csv("datasets/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("datasets/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "Parch", "SibSp"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

y = np.array([y]).T
X = np.array(X)

X_train = X[:(int(len(X)*0.75)), :]
X_val = X[(int(len(X)*0.75)):, :]
y_train = y[:(int(len(X)*0.75)), :]
y_val = y[(int(len(X)*0.75)):, :]
X_test = np.array(X_test)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()
X_test = torch.from_numpy(X_test).float()

In [5]:
X_test.shape

torch.Size([418, 5])

In [6]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        return x

net = Net()

optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)

In [7]:
EPOCHS = 1000
BATCH_SIZE = 167

for epoch in range(EPOCHS+1):
    permutation = torch.randperm(X_train.size()[0])
    for i in range(0, X_train.size()[0], BATCH_SIZE):
        net.zero_grad()
        ind = permutation[i:i+1]
        batch_x, batch_y = X_train[ind], y_train[ind]
        output = net(batch_x.view(-1, 5))
        loss = F.binary_cross_entropy(output, batch_y)
        loss.backward()
        optimizer.step()
    if epoch%100 == 0:
        print(epoch, loss)

/usr/lib/python3.9/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0 tensor(0.5346, grad_fn=<BinaryCrossEntropyBackward>)
100 tensor(0.6069, grad_fn=<BinaryCrossEntropyBackward>)
200 tensor(0.3976, grad_fn=<BinaryCrossEntropyBackward>)
300 tensor(1.1458, grad_fn=<BinaryCrossEntropyBackward>)
400 tensor(2.4356, grad_fn=<BinaryCrossEntropyBackward>)
500 tensor(0.5209, grad_fn=<BinaryCrossEntropyBackward>)
600 tensor(0.1748, grad_fn=<BinaryCrossEntropyBackward>)
700 tensor(0.1229, grad_fn=<BinaryCrossEntropyBackward>)
800 tensor(1.9597, grad_fn=<BinaryCrossEntropyBackward>)
900 tensor(0.1463, grad_fn=<BinaryCrossEntropyBackward>)
1000 tensor(0.1154, grad_fn=<BinaryCrossEntropyBackward>)


In [8]:
train_correct = 0
train_total = 0

with torch.no_grad():
    output = net(X_train.view(-1, 5))
    for idx, i in enumerate(output):
        if torch.round(i) == y_train[idx]:
            train_correct +=1
        train_total += 1

print("Accuracy:", round(train_correct/train_total*100, 2))

Accuracy: 80.39


In [9]:
test_correct = 0
test_total = 0

with torch.no_grad():
    output = net(X_val.view(-1, 5))
    for idx, i in enumerate(output):
        if torch.round(i) == y_val[idx]:
            test_correct +=1
        test_total += 1

print("Accuracy:", round(test_correct/test_total*100, 2))

Accuracy: 80.27


In [10]:
with torch.no_grad():
    output = net(X_test.view(-1, 5))

predictions = np.squeeze(torch.round(output))

In [11]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions.int()})
output.to_csv('titanic.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
